In [1]:
#%load_ext tensorboard
import os
import sys
sys.path.append(f"{os.environ['base']}Tools")

import btk
import cv2 as cv
import dataset_generators as dgen
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)
plt.style.use(f"{os.environ['style']}")


In [14]:
#trainx, trainy, valx, valy = btk.depickler('trainx-1643703742', 'trainy-1643703742', 'valx-1643703742', 'valy-1643703742', 'ocr')
#trainx, trainy, valx, valy = btk.depickler('trainx-1643708634', 'trainy-1643708634', 'valx-1643708634', 'valy-1643708634', 'ocr')

""" trainx, trainy = dgen.gen_chars(630000, dgen.allchars, dgen.trainfonts, True)
valx, valy = dgen.gen_chars(225000, dgen.allchars, dgen.evalfonts) """

""" trainx, trainy = dgen.gen_chars(315000, dgen.hardc, dgen.trainfonts, True)
valx, valy = dgen.gen_chars(110000, dgen.hardc, dgen.evalfonts, False) """

""" trainy = case_fix(trainy, dgen.allchars)
valy = case_fix(valy, dgen.allchars) """

#btk.pickle_set(trainx, trainy, valx, valy, 'ocr')

' trainy = case_fix(trainy, dgen.allchars)\nvaly = case_fix(valy, dgen.allchars) '

In [ ]:
#Character Classifier with CNN

imgent = btk.DataGen(trainx, trainy, 192)
imgenv = btk.DataGen(valx, valy, 64)


opti = keras.optimizers.Adam(learning_rate=0.0005)
log_dir = "tblogs/ccls-l/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tbcall = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
estop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=24, restore_best_weights=True)


inlay = keras.Input(shape=(64, 64, 1))
x = layers.Rescaling(1.0/255)(inlay)
x = layers.Conv2D(32, 7, padding='same', activation='relu')(x)
x = layers.Conv2D(32, 7, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.08)(x)
x = layers.Conv2D(64, 5, padding='same', activation='relu')(x)
x = layers.Conv2D(64, 5, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.08)(x)
x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.08)(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001))(x)
x = layers.Dropout(0.32)(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001))(x)
outlay = layers.Dense(54, activation='softmax')(x)


ccls = keras.Model(inlay, outlay, name='text_class')
ccls.compile(
    optimizer=opti,
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)
history = ccls.fit(
    x = imgent,
    validation_data=imgenv,
    epochs=200,
    steps_per_epoch=16,
    validation_steps=16,
    callbacks=[estop, tbcall],
    verbose=0
)

hist = pd.DataFrame(history.history)
hist[['categorical_accuracy', 'val_categorical_accuracy']].plot()
display(hist['val_categorical_accuracy'].max())
display(hist['val_loss'].min())

In [23]:
def case_fix(yar, superset):
    case_dic = {superset.index(x): superset.index(dgen.icase_chars[i]) for i, x in enumerate(dgen.case_chars)}
    for i, x in enumerate(yar):
        if x.argmax() in case_dic:
            tmp = x.argmax()
            yar[i][tmp] = 0
            yar[i][case_dic.get(tmp)] = 1
    return np.delete(yar, sorted(case_dic, reverse=True), 1)

ccls = tf.keras.models.load_model('models\\ccls-h')
#ccls.save('models\\ccls-h')
#ccls.summary()

In [ ]:
#Test classifier on images from segmentation module
ccls = tf.keras.models.load_model('models\\ccls-h')

with open('segset2', 'rb') as pkf:
    segset = btk.pickle.load(pkf)

csets = []
for x in segset:
    tmpset = []
    for y in x:
        image = np.array(Image.new(mode='L', color=15, size=(64, 64)))
        if y.size[0] > 32:
            pass
        else:
            im = y.resize((y.size[0] * 2, y.size[1] * 2))
            image[:, int(32 - y.size[0]):int(32 + y.size[0])] = im
            tmpset.append(image)
    csets.append(np.array(tmpset))


""" sharp = csets[0]
sharp = [cv.GaussianBlur(x, (3, 3), 5) for x in csets[0]]
sharp = [cv.GaussianBlur(x, (3, 3), 5) for x in sharp]
sharp = [cv.filter2D(src=x, ddepth=-1, kernel=btk.sharpenk) for x in sharp]
sharp = [cv.GaussianBlur(x, (3, 3), 5) for x in sharp]
sharp = [cv.filter2D(src=x, ddepth=-1, kernel=btk.sharpenk) for x in sharp]
sharp = [cv.GaussianBlur(x, (3, 3), 5) for x in sharp]
sharp = [cv.filter2D(src=x, ddepth=-1, kernel=btk.sharpenk) for x in sharp]
sharp = [cv.GaussianBlur(x, (3, 3), 5) for x in sharp]
sharp = [cv.filter2D(src=x, ddepth=-1, kernel=btk.sharpenk) for x in sharp]

sharp = np.array(sharp) """

ctest = ccls.predict(csets[0])

for i, x in enumerate(csets[0]):
    display(Image.fromarray(x))
    display(dgen.chars1[ctest[i].argmax()])

In [ ]:
#Test classifier on generated images or test generator
tx, ty = dgen.gen_chars(15, dgen.hardc, dgen.trainfonts, True)
preds = iocls.predict(tx)

for i, x in enumerate(tx):
    display(Image.fromarray(x))
    display(dgen.hardc[preds[i].argmax()])
    for j, y in enumerate(ty[i]):
        if y == 1:
            display(dgen.hardc[j])


In [ ]:
#Testing generated data vs model

tx, ty = dgen.gen_chars(5000, dgen.hardc, dgen.trainfonts, False, dgen.allchars)
ty = case_fix(ty, dgen.allchars)
preds = ccls.predict(tx)

guess = [dgen.chars1[preds[i].argmax()] for i in range(len(tx) - 1)]
real = [dgen.chars1[ty[i].argmax()] for i in range(len(ty) - 1)]

conf = confusion_matrix(y_true=real, y_pred=guess, labels=dgen.chars1)

fig, ax = plt.subplots()
fig.set_size_inches(24, 24)
im = ax.imshow(conf)

ax.set_xticks(np.arange(len(dgen.chars1)), labels=list(dgen.chars1))
ax.set_yticks(np.arange(len(dgen.chars1)), labels=list(dgen.chars1))
for i in range(len(dgen.chars1)):
    for j in range(len(dgen.chars1)):
        text = ax.text(j, i, conf[i, j], ha='center', va='center', color='w')

fig.tight_layout()
plt.show()

In [ ]:
correct = 0
incorrect = []

for i, x in enumerate(guess):
    if x == real[i]:
        correct += 1
    else:
        incorrect.append((x, real[i]))

    
display(f'{correct / 10000 * 100}% accuract')
display(btk.count(incorrect))

In [ ]:
for i in range(125, 175):
    display(Image.fromarray(trainx[i]))
    for j, y in enumerate(trainy[i]):
        if y == 1:
            display(dgen.chars1[j])